# Model Training and Testing

In this file, we train each of our models on the exact same training dataset, and test them
on the same testing dataset. This allows us to judge their performance, strengths, and
weaknesses using a comparable measure. We hope that this will, in conjunction with our
hypotheses, help us understand the merits of each model, and be more informed in future
instances of model selection.

## Imports

In [ ]:
import pandas as pd
import numpy as np
import sklearn as sk
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import precision_recall_fscore_support as class_metrics
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.gaussian_process.kernels import RBF, WhiteKernel
import matplotlib.pyplot as plt
import seaborn as sns
import regex as re
import pickle

## Create the Dataset

First, we separate out the testing and training dataset for each of our models to train on.
In this case, we decided against k-cross validation due to the sheer size of our dataset.
The accuracy of our models should be representative of their performance on future data as
well, since there is such a large quantity of training data.

Our dataset does have a few issues: several of the variables have classes which occur very infrequently.
This may bias our results, since we don't know much about how that variable affects heart disease relative to the other
classes. Therefore, we may need to stratify our data to get more accurate results.
Additionally, the columns **GenHealth** and **Stroke** have the potential to bias our results.
These factors are much more likely to occur due to heart disease, but may not be a good predictor
of its future fruition. Therefore, to deal with each of these issues, we will make separate versions of our dataset for
each of these cases: **Normal**, **Stratified**, and **No Health Indicators**.

In [ ]:
# Import the dataset
data = pd.read_csv('../data/raw_data.csv')

# Create a lists of each type of feature
nominal_features = ['Smoking', 'AlcoholDrinking', 'Stroke', 'DiffWalking', 'Sex', 'Race', 'Diabetic',
                  'PhysicalActivity', 'Asthma', 'KidneyDisease', 'SkinCancer']
ordinal_features = ['AgeCategory', 'GenHealth']
continuous_features = ['BMI', 'PhysicalHealth', 'MentalHealth', 'SleepTime']

# region Convert the nominal features into one-hot encodings

# Create a One-Hot Encoder
encoder = OneHotEncoder()

# For each nominal feature...
for feature in nominal_features:

    # Get an encoded version
    encoded_feature = pd.DataFrame(encoder.fit_transform(data[[feature]]).toarray(),
                                   columns=[f'{feature}_{f_class}' for f_class in data[feature].unique()])

    # Remove the old feature from the data
    data = data.drop(feature, axis=1)

    # Add the encoded feature to the data
    data = data.join(encoded_feature)

# endregion Convert the nominal features into one-hot encodings

# region Convert the ordinal features into labels

# For each nominal feature...
for feature in ordinal_features:

    # Replace the old feature with an encoded feature
    data[feature] = data[feature].astype('category').cat.codes

# endregion Convert the ordinal features into labels

# Convert the output column to be numerical
data['HeartDisease'] = data['HeartDisease'].astype('category').cat.codes

# Get the X and y of the dataset
data_X = data.iloc[:,1:]
data_y = data.loc[:,'HeartDisease']

# Display the head of the data
data.head()
print("Rows:    ", data.shape[0])
print("Columns: ", data.shape[1])

In [ ]:
# Get the Normal test-train split
normal_train_X, normal_test_X, normal_train_y, normal_test_y = train_test_split(data_X, data_y, train_size=2/3,
                                                                                random_state=42)

# Get the Stratified test-train split
stratified_train_X, stratified_test_X, stratified_train_y, stratified_test_y = train_test_split(data_X, data_y,
                                                                                                train_size=2/3,
                                                                                                stratify=data_y,
                                                                                                random_state=42)
# Get the No Health test-train split
dropped_features = [feature for feature in data.columns if ('GenHealth' in feature or 'Stroke' in feature) and feature in data_X.columns]
nohealth_train_X, nohealth_test_X, nohealth_train_y, nohealth_test_y = train_test_split(data_X.drop(dropped_features,
                                                                                                    axis=1),
                                                                                        data_y, train_size=2/3,
                                                                                        random_state=42)

## Dataset Tuning

Now that we know have our datasets, we need to find which we want to use. To do that, we can introduce a simple model
which we will use as a measure. By comparing the results of the model trained on each of the different datasets, we
can get a sense of how the changes impact performance, and potentially help us intuit whether any bias is introduced by
our data. In specific, we are looking to see if the dropped data columns or the stratification make the dataset perform
better or worse.

The model we will be using for this is **Logistic Regression**, since this model is very simple and fast to train.

In [ ]:
# Create a standard scalar and one-hot-encoding pipeline for each dataset
normal_pipe = Pipeline([('scalar', StandardScaler()), ('lr', LogisticRegression())])
stratified_pipe = Pipeline([('scalar', StandardScaler()), ('lr', LogisticRegression())])
nohealth_pipe = Pipeline([('scalar', StandardScaler()), ('lr', LogisticRegression())])

# Train each logistic regression on its dataset
normal_pipe.fit(normal_train_X, normal_train_y)
stratified_pipe.fit(stratified_train_X, stratified_train_y)
nohealth_pipe.fit(nohealth_train_X, nohealth_train_y)

# Make predictions with each logistic regression
normal_prediction = normal_pipe.predict(normal_test_X)
stratified_prediction = stratified_pipe.predict(stratified_test_X)
nohealth_prediction = nohealth_pipe.predict(nohealth_test_X)

# Find the score of each regression
normal_precision, normal_recall, normal_f, _ = class_metrics(normal_test_y, normal_prediction)
stratified_precision, stratified_recall, stratified_f, _ = class_metrics(stratified_test_y, stratified_prediction)
nohealth_precision, nohealth_recall, nohealth_f, _ = class_metrics(nohealth_test_y, nohealth_prediction)

# Display the score of each regression
print("Normal Dataset LR:")
print(f"Precision: {normal_precision[1]}")
print(f"Recall:    {normal_recall[1]}")
print(f"F1 Score:  {normal_f[1]}")
print()
print("Stratified Dataset LR:")
print(f"Precision: {stratified_precision[1]}")
print(f"Recall:    {stratified_recall[1]}")
print(f"F1 Score:  {stratified_f[1]}")
print()
print("No Health Dataset LR:")
print(f"Precision: {nohealth_precision[1]}")
print(f"Recall:    {nohealth_recall[1]}")
print(f"F1 Score:  {nohealth_f[1]}")

As we can see, the stratified dataset performed marginally better than the normal dataset, whereas the No Health
indicators dataset performed moderately worse. If were really trying to predict heart disease, this would indicate to
us that using these columns may be unhelpful for our predictions. However, since that is not the case and since we have
the data, we are going to use it anyways, and keep with the stratified dataset.

However, some of our models may need a smaller subset of the data, such as the Gaussian Process Classifier. This is
because they have high memory demands which scale with the data.

Below, let's do two things: Relabel the stratified dataset we'll be using to be more simple, and create a subset of that
dataset for models which need less data

In [ ]:
# Rename the stratified dataset
train_X, test_X, train_y, test_y = stratified_train_X, stratified_test_X, stratified_train_y, stratified_test_y

# Get a smaller subset of the dataset
data_subset = data.sample(5000, random_state=42)
data_sub_X = data_subset.iloc[:,1:]
data_sub_y = data_subset.loc[:,'HeartDisease']

# Get the Stratified test-train split
train_sub_X, test_sub_X, train_sub_y, test_sub_y = train_test_split(data_sub_X, data_sub_y, train_size=2/3,
                                                                    stratify=data_sub_y, random_state=42)

# Model Training

Now, we need to train each of our models on the stratified data. Since our training dataset is so large (around
200,000 samples), we do not need to cross validate our results for each model - they should be representative of true
performance relative to one another.

Let's start by defining some methods that makes sure that each of our models is trained the same general way.

In [ ]:
def train_model(model, parameter_dict, dataset_train_x, dataset_train_y):
    """This function is used to the find the best model using Cross Validating grid search.
    It also applies a standard scaling pipeline to the data.

    :param model: An SK Learn model to train
    :param parameter_dict: A dictionary of parameters to optimize using grid search. Keys are non-pipe parameters.
    :param dataset_train_x: The X values of the training dataset
    :param dataset_train_y: The y values of the training dataset
    :return best_model: The best model found by the grid search, retrained on all of the data
    :return search: The grid search object
    """

    # Create the pipeline
    pipe = Pipeline([('scalar', StandardScaler()), ('model', model)])

    # Find the parameter grid
    parameter_grid = {'model__' + param: parameter_dict[param] for param in parameter_dict}

    # Get the GridSearch object
    search = GridSearchCV(pipe, parameter_grid, cv=3, n_jobs=-1, verbose=100, scoring='f1')

    # Search for the best model
    search.fit(dataset_train_x, dataset_train_y)

    # Train the best model found
    best_model = Pipeline([('scalar', StandardScaler()), ('model', model)]).set_params(**search.best_params_)
    best_model.fit(dataset_train_x, dataset_train_y)

    # Return the grid search object
    return best_model, search


def display_results(model_name, best_model, grid_search, dataset_train_x, dataset_train_y,
                    dataset_test_x, dataset_test_y):
    """This function displays the results of the grid search both via tables and graphically.

    :param model_name: The name of the SK Learn model trained
    :param best_model: The best model found by the grid search
    :param grid_search: The grid search object which optimized the model
    :param dataset_train_x: The X values of the training dataset
    :param dataset_train_y: The y values of the training dataset
    :param dataset_test_x: The X values of the training dataset
    :param dataset_test_y: The y values of the training dataset
    """

    # Add spacing
    print("\n")

    # Start by displaying the trend of the data over each of the parameters
    for param in grid_search.best_params_:

        # This dict stores the mean performance for each parameter
        param_dict = {}

        # For the parameter value for each grid search iteration
        for p_ind, p in enumerate(grid_search.cv_results_[f'param_{param}']):

            # Get P's name
            pname = str(p)

            # If the parameter is not in the param dictionary, add it
            if pname not in param_dict: param_dict[pname] = (1, grid_search.cv_results_['mean_test_score'][p_ind])

            # Otherwise, summate them
            else: param_dict[pname] = (param_dict[pname][0] + 1,
                                   param_dict[pname][1] + grid_search.cv_results_['mean_test_score'][p_ind])

        # Finally, find the average value for each parameter value across all grid search iteration
        for p in param_dict:
            pname = str(p)
            param_dict[pname] = param_dict[pname][1] / param_dict[pname][0]

        # If the data is numeric, draw a lineplot
        if type(list(param_dict.keys())[0]) is not str:
            xtick_enum = range(len(list(param_dict.keys())))
            ax = sns.lineplot(x=xtick_enum, y=param_dict.values())
            ax.set_xticks(xtick_enum, list(param_dict.keys()))

        # otherwise, draw a barplot
        else: ax = sns.barplot(x=list(param_dict.keys()), y=list(param_dict.values()))

        # Since we only care about the differences between models, zoom in the plot to see the differences
        max_val = max(param_dict.values())
        min_val = min(param_dict.values())
        range_val = max_val - min_val
        plt.ylim(min_val - range_val*0.1, max_val + range_val*0.1)

        # Add plot aesthetics
        ax.set_xlabel(param[7:].capitalize() + ' Value')
        ax.set_ylabel('F1 Score')
        plt.title(f'F1 Score over {param[7:].capitalize()} Value')
        plt.show()

    # Calculate the performance of the best model
    model_train_pred = best_model.predict(dataset_train_x)
    model_test_pred = best_model.predict(dataset_test_x)

    # Calculate the performance metrics of the best model
    model_train_precision, model_train_recall, model_train_f1, _ = class_metrics(dataset_train_y, model_train_pred,
                                                                                 zero_division=0)
    model_test_precision, model_test_recall, model_test_f1, _ = class_metrics(dataset_test_y, model_test_pred,
                                                                              zero_division=0)

    # Display which parameters were best
    print("The best parameters were:")
    for param in grid_search.best_params_:
        print(re.search(r'model__(.*)', param).group(1) + f": {grid_search.best_params_[param]}")
    print("\n")

    # Display the scores of the best model
    print(f"{model_name.title()} Prediction Metrics:")
    print(f"Train Precision: {model_train_precision[1]}")
    print(f"Train Recall:    {model_train_recall[1]}")
    print(f"Train F1 Score:  {model_train_f1[1]}")
    print(f"Test Precision:  {model_test_precision[1]}")
    print(f"Test Recall:     {model_test_recall[1]}")
    print(f"Test F1 Score:   {model_test_f1[1]}")


def save_model(model_name, grid_search):
    """Save the best model's parameters.

    :param model_name: The name of the SK Learn model trained
    :param grid_search: The grid search object which optimized the model
    """

    pickle.dump(grid_search.best_params_, open(f'../models/{model_name}.pkl', 'wb'))

## Baseline Model

A good place to start is to make a baseline. The most simple baseline model we can use is one that always guesses that there is heart disease. Let's see how this performs, in order to compare it to our other models.

In [ ]:
# Create a dataframe full of 'Yes' Heart Disease Predictions
baseline_train_pred = pd.Series([1] * train_y.shape[0])
baseline_test_pred = pd.Series([1] * test_y.shape[0])

# Find the baseline classification metrics
baseline_train_precision, baseline_train_recall, baseline_train_f, _ = class_metrics(train_y,
                                                                                     baseline_train_pred,
                                                                                     zero_division=0)
baseline_test_precision, baseline_test_recall, baseline_test_f, _ = class_metrics(test_y,
                                                                                  baseline_test_pred,
                                                                                  zero_division=0)

# Display the score of each regression
print("Baseline Model Prediction Metrics:")
print(f"Train Precision: {baseline_train_precision[1]}")
print(f"Train Recall:    {baseline_train_recall[1]}")
print(f"Train F1 Score:  {baseline_train_f[1]}")
print(f"Test Precision:  {baseline_test_precision[1]}")
print(f"Test Recall:     {baseline_test_recall[1]}")
print(f"Test F1 Score:   {baseline_test_f[1]}")


As we can see, our model does pretty poorly overall. Although it does correctly label all occurrences of heart disease,
it fails to label any of the negative cases. Our F Score to beat comes out to be **0.159**.

## Gaussian Process Classification
*By Will Sumerfield*

### Gaussian Process Classification Hypothesis

Given that GPC models perform very well on datasets with a good spread over the dataspace, and that our dataset is very
large, I predict that the Gaussian Process Classifier model will perform very well, if I can find a good kernel
function for the data. However, I expect that this will be a very difficult process, given that there are many
columns of our data.

Additionally, I may need to use smaller subsamples of the data to train the GPC, given that GPCs take up **$O(n^2)$**
space, and take the same training time. I expect this model to be among, if not the best model we employ.

In [ ]:
# Create a list of potential parameters
gpc_params = {'kernel': [RBF(), WhiteKernel() + RBF()]}

# Train the model, and find the best model from the provided inputs
best_model, grid_search = train_model(GaussianProcessClassifier(random_state=42, copy_X_train=False), gpc_params,
                                      train_sub_X, train_sub_y)

# Display the results
display_results("Gaussian Process Classifier", best_model, grid_search, train_sub_X, train_sub_y, test_sub_X, test_sub_y)

# Save the model
save_model("Gaussian Process Classifier", grid_search)

### Gaussian Process Classification Analysis

Put a summary of your model's performance in this spot!

## Copy-Paste all the GPC stuff here! Don't run the entire notebook, or you will train each model. That will take ages...
